In [1]:
import requests
from bs4 import BeautifulSoup
import time


from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

import pandas as pd
import numpy as np

import random


In [2]:
import json
import ssl
import time

from urllib.request import urlopen


In [3]:
#input

userType_ls = [1,1,1,1,1,1,
           1,1,1,1,1,1,1,
           2,2,2,2,2,2,
           2,2,2,2] #1. adult 2. children/adolescence

# 1: mental health. 2: drug and alcohol
serviceType_ls = [1,1,1,1,1,1, #adult +  mental health
               2,2,2,2,2,2,2, # adult + drug and alcohol
               1,1,1,1,1,1, #chidren +  mental health
                 2,2,2,2] #childern + drug and alcohol
#### adult
service11 = [1751,1722, 1746, 1766, 1758, 1763]  #6
service11_key = ["Outpatient Therapy",
               "Blended case management",  
               "Intensive case management",
               "Resource coordination", 
               "Neuropsychological testing",
              "Psychological testing"]

service12 = [1737, 1731, 1732, 1725,1733, 1727, 1739]  #7
service12_key = ["Outpatient Therapy", 
                "Intensive outpatient",
               "Level of care assessment",
               "Buprenorphine treatment",
               "Methadone maintenance",
               "Certified recovery specialist",
               "Recovery support"]

#### children/adolescence
service21 = service11 #the same as adult
service21_key = service11_key

service22  = [1737, 1731, 1732, 1727] #4
service22_key = ["Outpatient Therapy", 
                "Intensive outpatient",
               "Level of care assessment",
                "Certified recovery specialist"]


##### Aggregate
service_ls = np.concatenate((service11, service12, service21, service22))
service_key_ls = np.concatenate((service11_key, service12_key, service21_key, service22_key))

print(len(service_ls) == len(service_key_ls) == len(serviceType_ls) == len(userType_ls))


True


In [4]:
search_grid = {
    "service" : service_ls,
    "service_key": service_key_ls,
    "serviceType": serviceType_ls,
    "userType_ls" : userType_ls
}
search_grid = pd.DataFrame(search_grid)
search_grid

,service,service_key,serviceType,userType_ls
0,1751,Outpatient Therapy,1,1
1,1722,Blended case management,1,1
2,1746,Intensive case management,1,1
3,1766,Resource coordination,1,1
4,1758,Neuropsychological testing,1,1
5,1763,Psychological testing,1,1
6,1737,Outpatient Therapy,2,1
7,1731,Intensive outpatient,2,1
8,1732,Level of care assessment,2,1
9,1725,Buprenorphine treatment,2,1


In [44]:
start_time = time.time()

PRACTICEName = []
PRACTICEPhone = []
PRACTICEfullAddress = []
PRACTICEfullAddressSingleLine = []
PRACTICEnewPatientsTF = []
latitude = []
longitude = []
countyName = []
provider_name =[]
genderLabel =[]
provider_discipline = []
affiliation = []
boardCertification = []
language = []
providerPhone = []
MDnewPatientsTF = []
serviceType = []
service = []
userType = []
radius = []
searchIndex = []

zip = "15261"
distance = 20

page = 1 #start with page 1
print("Current page:", page)


#for search_index in range(len(service_ls)-1):
#for search_index in range(0,len(service_ls)-1,1):
for search_index in range(4,10):
    #time.sleep(10)
    print('***Current searching index:', search_index)
    serviceType_i = serviceType_ls[search_index] 
    userType_i = userType_ls[search_index]
    service_i = service_ls[search_index]

    api_base_url ="https://members.ccbh.com/ProviderDirectory/Api/Search?serviceTypeID={serviceType_i}&userTypeID={userType_i}&serviceIDs={service_i}&zip={zip}&distance={distance}&language=English&genderID=0&newPatientsTF=&countyID=2&page={page}&contractID=1&entityName="

    while True:
        api_url = api_base_url.format(zip=zip, 
                                      page=page, 
                                      serviceType_i = serviceType_i,userType_i= userType_i, service_i= service_i, 
                                      distance = distance)
        with urlopen(api_url) as f:
            resp = json.load(f)
        #print('Success: we got the page: ',resp)

        if len(resp["data"]["entities"]) == 0 : #put this in front of the : to test "or page >= 2"
            time.sleep(random.randint(5, 7))

            print("Empty page. Go to next searching index.")
            PRACTICEName.append(False)
            PRACTICEPhone.append(False)
            PRACTICEfullAddress.append(False)
            PRACTICEfullAddressSingleLine.append(False)
            PRACTICEnewPatientsTF.append(False)
            latitude.append(False)
            longitude.append(False)
            countyName.append(False)
            provider_name.append(False)
            genderLabel.append(False)
            provider_discipline.append(False)
            affiliation.append(False)
            boardCertification.append(False)
            language.append(False)
            providerPhone.append(False)
            MDnewPatientsTF.append(False)
            serviceType.append(serviceType_i)
            service.append(service_i)
            userType.append(userType_i)
            radius.append(distance)
            searchIndex.append(search_index)
            break
            
        
        for practice in resp["data"]["entities"]:
            if practice["doctors"] is None: # Under practice, there are no doctors
                print("!!!Warning: Under practice, there are no doctors.")
                print("***Scrape practice name:", practice["providerName"])
                PRACTICEName.append(practice["providerName"])
                PRACTICEPhone.append(practice["providerPhone"])
                PRACTICEfullAddress.append(practice["fullAddress"])
                PRACTICEfullAddressSingleLine.append(practice["fullAddressSingleLine"])
                PRACTICEnewPatientsTF.append(practice["newPatientsTF"])
                latitude.append(practice["latitude"])
                longitude.append(practice["longitude"])
                countyName.append(practice["countyName"])
                provider_name.append(False)
                genderLabel.append(False)
                provider_discipline.append(False)
                affiliation.append(False)
                boardCertification.append(False)
                language.append(False)
                providerPhone.append(False)
                MDnewPatientsTF.append(False)
                serviceType.append(serviceType_i)
                service.append(service_i)
                userType.append(userType_i)
                radius.append(distance)
                searchIndex.append(search_index)
            else: # Under practice, there are listed doctors
                for doctor in practice["doctors"]:
                    print("***Scrape provider name:", doctor["doctorName"])
                    PRACTICEName.append(practice["providerName"])
                    PRACTICEPhone.append(practice["providerPhone"])
                    PRACTICEfullAddress.append(practice["fullAddress"])
                    PRACTICEfullAddressSingleLine.append(practice["fullAddressSingleLine"])
                    PRACTICEnewPatientsTF.append(practice["newPatientsTF"])
                    latitude.append(practice["latitude"])
                    longitude.append(practice["longitude"])
                    countyName.append(practice["countyName"])
                    provider_name.append(doctor["doctorName"])
                    genderLabel.append(doctor["genderLabel"])
                    provider_discipline.append(doctor["disciplineName"])
                    affiliation.append(doctor["affiliation"])
                    boardCertification.append(doctor["boardCertification"])
                    language.append(doctor["language"])
                    providerPhone.append(doctor["doctorPhone"])
                    MDnewPatientsTF.append(doctor["newPatientsTF"])
                    serviceType.append(serviceType_i)
                    service.append(service_i)
                    userType.append(userType_i)
                    radius.append(distance)
                    searchIndex.append(search_index)
        time.sleep(random.randint(10, 12))
        page += 1
        print("Current page:", page)
    
print("--- %s minutes ---" ,(time.time() - start_time)/60)

data = {
        'PRACTICEName': PRACTICEName,
        'PRACTICEPhone': PRACTICEPhone,
        'PRACTICEfullAddress':PRACTICEfullAddress,
        'PRACTICEfullAddressSingleLine':PRACTICEfullAddressSingleLine,
        'PRACTICEnewPatientsTF' :PRACTICEnewPatientsTF,
        'latitude' : latitude,
        'longitude' :longitude,
        'countyName' :countyName,
        'provider_name':provider_name,
        'genderLabel' :genderLabel,
        'provider_discipline':provider_discipline,
        'affiliation':affiliation,
        'boardCertification':boardCertification,
        'language':language,
        'providerPhone' :providerPhone,
        'MDnewPatientsTF':MDnewPatientsTF,
        'serviceType': serviceType,
        'service': service,
        'userType': userType,
        'radius': radius,
        'searchIndex': searchIndex
        }


df = pd.DataFrame(data)
df = pd.DataFrame.from_dict(data)
df.to_csv('output/provider_list_20miles_i4.csv', index = 0)

Current page: 1
***Current searching index: 4
!!!Warning: Under practice, there are no doctors.
***Scrape practice name: UPMC Western Psychiatric Hospital
!!!Warning: Under practice, there are no doctors.
***Scrape practice name: University of Pittsburgh Physicians
***Scrape provider name: Mates, Jenna, PsyD
***Scrape provider name: Simonds, Rachel, PhD
***Scrape provider name: Coburn, P. Christopher, PhD
***Scrape provider name: Bowen, Susan, PhD
***Scrape provider name: Collin Dilmore, V. Terri, PhD
***Scrape provider name: Coufal, Robert, PhD
***Scrape provider name: Healey, Sean, PsyD
***Scrape provider name: Simonds, Rachel, PhD
***Scrape provider name: Gruner, Daniel, PhD
***Scrape provider name: Hoover, Wayne, PhD
***Scrape provider name: Newman, Lawrence, PhD
Current page: 2
***Scrape provider name: Saul, Daniel, PsyD
***Scrape provider name: Schramke, Carol, PhD
!!!Warning: Under practice, there are no doctors.
***Scrape practice name: Allegheny Clinic
!!!Warning: Under practi

# Job 2

In [40]:
start_time = time.time()

PRACTICEName = []
PRACTICEPhone = []
PRACTICEfullAddress = []
PRACTICEfullAddressSingleLine = []
PRACTICEnewPatientsTF = []
latitude = []
longitude = []
countyName = []
provider_name =[]
genderLabel =[]
provider_discipline = []
affiliation = []
boardCertification = []
language = []
providerPhone = []
MDnewPatientsTF = []
serviceType = []
service = []
userType = []
radius = []
searchIndex = []

zip = "15261"
distance = 20

page = 1 #start with page 1
print("Current page:", page)


#for search_index in range(len(service_ls)-1):
#for search_index in range(0,len(service_ls)-1,1):
for search_index in range(4,5):
    #time.sleep(10)
    print('***Current searching index:', search_index)
    serviceType_i = serviceType_ls[search_index] 
    userType_i = userType_ls[search_index]
    service_i = service_ls[search_index]

    api_base_url ="https://members.ccbh.com/ProviderDirectory/Api/Search?serviceTypeID={serviceType_i}&userTypeID={userType_i}&serviceIDs={service_i}&zip={zip}&distance={distance}&language=English&genderID=0&newPatientsTF=&countyID=2&page={page}&contractID=1&entityName="

    while True:
        api_url = api_base_url.format(zip=zip, 
                                      page=page, 
                                      serviceType_i = serviceType_i,userType_i= userType_i, service_i= service_i, 
                                      distance = distance)
        with urlopen(api_url) as f:
            resp = json.load(f)
        #print('Success: we got the page: ',resp)

        if len(resp["data"]["entities"]) == 0 : #put this in front of the : to test "or page >= 2"
            time.sleep(random.randint(5, 7))

            print("Empty page. Go to next searching index.")
            PRACTICEName.append(False)
            PRACTICEPhone.append(False)
            PRACTICEfullAddress.append(False)
            PRACTICEfullAddressSingleLine.append(False)
            PRACTICEnewPatientsTF.append(False)
            latitude.append(False)
            longitude.append(False)
            countyName.append(False)
            provider_name.append(False)
            genderLabel.append(False)
            provider_discipline.append(False)
            affiliation.append(False)
            boardCertification.append(False)
            language.append(False)
            providerPhone.append(False)
            MDnewPatientsTF.append(False)
            serviceType.append(serviceType_i)
            service.append(service_i)
            userType.append(userType_i)
            radius.append(distance)
            searchIndex.append(search_index)
            break
            
        
        for practice in resp["data"]["entities"]:
            if practice["doctors"] is None: # Under practice, there are no doctors
                print("!!!Warning: Under practice, there are no doctors.")
                print("***Scrape practice name:", practice["providerName"])
                PRACTICEName.append(practice["providerName"])
                PRACTICEPhone.append(practice["providerPhone"])
                PRACTICEfullAddress.append(practice["fullAddress"])
                PRACTICEfullAddressSingleLine.append(practice["fullAddressSingleLine"])
                PRACTICEnewPatientsTF.append(practice["newPatientsTF"])
                latitude.append(practice["latitude"])
                longitude.append(practice["longitude"])
                countyName.append(practice["countyName"])
                provider_name.append(False)
                genderLabel.append(False)
                provider_discipline.append(False)
                affiliation.append(False)
                boardCertification.append(False)
                language.append(False)
                providerPhone.append(False)
                MDnewPatientsTF.append(False)
                serviceType.append(serviceType_i)
                service.append(service_i)
                userType.append(userType_i)
                radius.append(distance)
                searchIndex.append(search_index)
            else: # Under practice, there are listed doctors
                for doctor in practice["doctors"]:
                    print("***Scrape provider name:", doctor["doctorName"])
                    PRACTICEName.append(practice["providerName"])
                    PRACTICEPhone.append(practice["providerPhone"])
                    PRACTICEfullAddress.append(practice["fullAddress"])
                    PRACTICEfullAddressSingleLine.append(practice["fullAddressSingleLine"])
                    PRACTICEnewPatientsTF.append(practice["newPatientsTF"])
                    latitude.append(practice["latitude"])
                    longitude.append(practice["longitude"])
                    countyName.append(practice["countyName"])
                    provider_name.append(doctor["doctorName"])
                    genderLabel.append(doctor["genderLabel"])
                    provider_discipline.append(doctor["disciplineName"])
                    affiliation.append(doctor["affiliation"])
                    boardCertification.append(doctor["boardCertification"])
                    language.append(doctor["language"])
                    providerPhone.append(doctor["doctorPhone"])
                    MDnewPatientsTF.append(doctor["newPatientsTF"])
                    serviceType.append(serviceType_i)
                    service.append(service_i)
                    userType.append(userType_i)
                    radius.append(distance)
                    searchIndex.append(search_index)
        time.sleep(random.randint(10, 12))
        page += 1
        print("Current page:", page)
    
print("--- %s minutes ---" ,(time.time() - start_time)/60)

data = {
        'PRACTICEName': PRACTICEName,
        'PRACTICEPhone': PRACTICEPhone,
        'PRACTICEfullAddress':PRACTICEfullAddress,
        'PRACTICEfullAddressSingleLine':PRACTICEfullAddressSingleLine,
        'PRACTICEnewPatientsTF' :PRACTICEnewPatientsTF,
        'latitude' : latitude,
        'longitude' :longitude,
        'countyName' :countyName,
        'provider_name':provider_name,
        'genderLabel' :genderLabel,
        'provider_discipline':provider_discipline,
        'affiliation':affiliation,
        'boardCertification':boardCertification,
        'language':language,
        'providerPhone' :providerPhone,
        'MDnewPatientsTF':MDnewPatientsTF,
        'serviceType': serviceType,
        'service': service,
        'userType': userType,
        'radius': radius,
        'searchIndex': searchIndex
        }

df = pd.DataFrame(data)

df = pd.DataFrame.from_dict(data)
df.to_csv('output/provider_list_20miles_i2.csv', index = 0)


Current page: 1
***Current searching index: 2
!!!Warning: Under practice, there are no doctors.
***Scrape practice name: Mercy Behavioral Health
!!!Warning: Under practice, there are no doctors.
***Scrape practice name: Milestone Centers, Inc.
!!!Warning: Under practice, there are no doctors.
***Scrape practice name: Turtle Creek Valley MH/MR, Inc.
!!!Warning: Under practice, there are no doctors.
***Scrape practice name: Sewickley Valley Hospital (Staunton Clinic)
Current page: 2
Empty page. Go to next searching index.
--- %s minutes --- 0.2942031979560852
